In [1]:
! pip install sacremoses

In [ ]:
from enum import Enum
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModelForMaskedLM

roberta_pipe = pipeline("fill-mask", model="FacebookAI/xlm-roberta-base")

herbert_model = AutoModelForMaskedLM.from_pretrained("allegro/herbert-base-cased")
herbert_tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
herbert_pipe = pipeline("fill-mask", model=herbert_model, tokenizer=herbert_tokenizer)

polbert_model = AutoModelForMaskedLM.from_pretrained("dkleczek/bert-base-polish-cased-v1")
polbert_tokenizer = AutoTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1")
polbert_pipe = pipeline("fill-mask", model=polbert_model, tokenizer=polbert_tokenizer)

papugapt_tokenizer = AutoTokenizer.from_pretrained("flax-community/papuGaPT2")
papugapt_model = AutoModelForCausalLM.from_pretrained("flax-community/papuGaPT2")
papugapt_model.config.pad_token_id = papugapt_model.config.eos_token_id
papugapt_pipe = pipeline("text-generation", model=papugapt_model, tokenizer=papugapt_tokenizer, device='cuda')


In [3]:
import os

results_dir_path = "./results"
os.makedirs(results_dir_path, exist_ok=True)

In [12]:
import warnings
from collections import Counter

n = 1000 # param for papugapt

class LLM(Enum):
    ROBERTA = 0
    HERBERT = 1
    POLBERT = 2
    PAPUGAPT = 3


def fill_sentence_with(sentence: str, model: LLM):
  if not "<mask>" in sentence:
    warnings.warn(f"<mask> not found in snetence: {sentence}")

  predictions = None

  if model == LLM.ROBERTA:
    predictions = roberta_pipe(sentence)

  elif model == LLM.HERBERT:
    predictions = herbert_pipe(sentence)

  elif model == LLM.POLBERT:
    # mask token change required
    polbert_sentence = sentence.replace("<mask>", "[MASK]")
    predictions = polbert_pipe(polbert_sentence)

  elif model == LLM.PAPUGAPT:
    # papugapt allows only to add items at the end of given sentence
    if 'mask' in sentence.split()[-1].lower():
      papugapt_sentence = ' '.join(sentence.split()[:-1])
      outputs = papugapt_pipe(papugapt_sentence, max_length=len(papugapt_tokenizer(papugapt_sentence)['input_ids']) + 1, num_return_sequences=n, truncation=True)
      words_to_count = [output['generated_text'].split()[-1] for output in outputs]
      counter = Counter(words_to_count)
      most_common_words = counter.most_common(5)
      words = [word for word, freq in most_common_words]
      scores = [freq/n for word, freq in most_common_words]
      # papugapt not returns scores explicite so scores for this model are calculated
      # as frequency/n for n one word ending propsitions

    else:
      words = ["" for _ in range(5)]
      scores = [0 for _ in range(5)]
      # empty string and 0 returned for integrity if mask not at the end

    return words, scores

  else:
    warnings.warn('Model not found')

  if predictions:
    words = [prediction['token_str'] for prediction in predictions]
    scores = [prediction['score'] for prediction in predictions]
    return words, scores

  return

W porównaniu zastosowane zostały cztery modele. Trzy z nich (Roberta, Herbert i Polbert) były używane z wykorzystaniem wbudowanej w bibliotekę transformers funkcji pipeline do uzupełniania luk. Za każdym razem pipeline z pipeline zwracany był słownik z wynikami, z którego do prezentacji wyników wyciągane były parametry 'token_str' i 'score'. 

Czwarty z modeli - PapuGaPT2 jest przygotowany z myślą o generacji tekstu i nie dał się łatwo zmapować do pipeline podobnego jak dla wcześniejszych modeli. Aby móc jakkolwiek porównać wyniki uzyskiwane przez ten model z innymi, wiele zdań posiada token maskujący na ostatnim miejscu. Model PapuGaPT2 był używany tylko z takimi zdaniami, a wynikowe zdanie miało mieć tylko jeden token więcej niż zdanie oryginalne. Ta procedura powtarzana była wiele razy (dla większości modeli 1000), a następnie predykcje z listy były zliczane - pięć najczęściej występujących predykcji było zwracanych, a jako odpowiednik 'score' traktowany był odsetek każdej z odpowiedzi w zbiorze zwróconych przez model.


In [5]:
import re

# for saving to file only
def remove_non_literals(sentence):
    return re.sub(r'[^a-zA-Z0-9ąćęłńóśźżĄĆĘŁŃÓŚŹŻ_]', '', sentence)

In [6]:
import pandas as pd
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)

def test_sequence_with_all_models(sentence):
  result_dict = {}
  for model in LLM:
    words, scores = fill_sentence_with(sentence, model)
    result_dict[f"{model.name}_words"] = words
    result_dict[f"{model.name}_scores"] = scores

  result_df = pd.DataFrame(result_dict)
  print(' >>> ' + sentence + ' <<<\n')
  print(result_df)
  print('\n' + '-'*100 + '\n')
  results_path = results_dir_path + '/' + remove_non_literals(sentence.lower().replace("<mask>", 'MASK').strip().replace(' ', '_')) + '.csv'
  result_df.to_csv(results_path, index=False)

In [7]:
import logging

logging.getLogger("transformers").setLevel(logging.ERROR)

In [8]:
sentences = ["<mask> jest stolicą Polski.", "Warszawa jest stolicą <mask>."]

for sentence in sentences:
  test_sequence_with_all_models(sentence)

 >>> <mask> jest stolicą Polski. <<<

  ROBERTA_words  ROBERTA_scores HERBERT_words  HERBERT_scores POLBERT_words  POLBERT_scores PAPUGAPT_words  PAPUGAPT_scores
0      Warszawa        0.372895      Warszawa        0.310033      Warszawa        0.187919                               0
1        Polska        0.183184        Kraków        0.270802        Miasto        0.075764                               0
2        Kraków        0.089364       Wrocław        0.056544        Poznań        0.074573                               0
3       Wrocław        0.036436        Poznań        0.049899        Kraków        0.053991                               0
4          Łódź        0.028232          Łódź        0.041426       Wrocław        0.051718                               0

----------------------------------------------------------------------------------------------------

 >>> Warszawa jest stolicą <mask>. <<<

  ROBERTA_words  ROBERTA_scores HERBERT_words  HERBERT_scores POLBERT_words

Już podstawowe pytanie uwidacznia niektóre problemy wybranych modeli językowych. Roberta, Herbert i Polbert są zgodne co do tego, że stolicą Polski jest Warszawa. 

Odwrócenie pytania powoduje jednak pewne trudności - Herbert i Polbert udzielają oczekiwanej odpowiedzi, według Roberty Warszawa jest stolicą kultury (to oraz odpowiedź "nauki" na drugim miejscu pozwala sądzić, że słowo "Warszawa" jest w tym modelu mocno powiązane z Pałacem Kultury i Nauki), a PapuGaPT2 w niemal połowie odpowiedzi twierdzi, że Warszawa jest stolicą województwa - jest to niewątpliwie prawda, ale jednak nie takiej odpowiedzi można było oczekiwać.


In [66]:
pl_cases = {
  "M": "W tym zespole na gitarze gra przystojny <mask>.",
  "D": "Znana francuska malarka jest autorką tego <mask>.",
  "C": "W czasie spaceru po kwiecistej łące zrobiłem zdjęcie pięknemu <mask>.",
  "B": "W księgarni znalazłam ciekawą <mask>.",
  "N": "Osoba zajmująca się wypiekaniem chleba jest z zawodu <mask>.",
  "Msc": "Kibice piłki nożnej rozmawiali o swoim ulubionym <mask>.",
  "W": "<mask>, pomóż mi rozwiązać ten problem!"
}

In [69]:
for pl_case, sentence in pl_cases.items():
  print(' >>> ' + pl_case + '. <<<\n')
  test_sequence_with_all_models(sentence)

 >>> M. <<<

 >>> W tym zespole na gitarze gra przystojny <mask>. <<<

  ROBERTA_words  ROBERTA_scores HERBERT_words  HERBERT_scores POLBERT_words  POLBERT_scores PAPUGAPT_words  PAPUGAPT_scores
0        zespół        0.053240     mężczyzna        0.162646       chłopak        0.118237    przystojny,            0.293
1         muzyk        0.046967       chłopak        0.039528         muzyk        0.092068              i            0.173
2         Dawid        0.029852         facet        0.021149     mężczyzna        0.070187        chłopak            0.103
3        Michał        0.029052            DJ        0.020006    perkusista        0.055205      wokalista            0.048
4          głos        0.024649         muzyk        0.017788    gitarzysta        0.041880      mężczyzna            0.034

----------------------------------------------------------------------------------------------------

 >>> D. <<<

 >>> Znana francuska malarka jest autorką tego <mask>. <<<

  ROBERTA

## Odmiana przez przypadki:
### Liczba pojedyncza:
- **Mianownik** - forma w zdecydowanej większości ok, sens pozostawia wiele do życzenia (Roberta - "Na gitarze gra przystojny ZESPÓŁ")
- **Dopełniacz** - forma w większości w porządku, sens też
- **Celownik** - modele mają problem ze znalezieniem właściwej formy - bezbłędny właściwie tylko Polbert, Roberta wszystkie źle, PapuGaPT2 w założonym jednym tokenie nie dał rady rozwinąć skrzydeł (może pozwalając mu dokończyć słowa dostalibyśmy całkiem dobre wyniki)
- **Biernik** - forma i sens dla wszystkich modeli w porządku
- **Narzędnik** - Herbert i Polbert z formą radzą sobie całkiem nieźle, gorzej z sensem - nauczyciele raczej rzadko pieką chleb w pracy
- **Miejscownik** - forma i sens dla najlepszych dopasowań w zasadzie ok, dla wszystkich modeli
- **Wołacz** - dodane wyrazy mają sens i są w poprawnej formie - Herbert okazuje się być bardzo religijnym modelem językowym


In [70]:
pl_cases_plural = {
  "M": "W tym zespole na gitarach grają dwaj przystojni <mask>.",
  "D": "Znana francuska malarka jest autorką tych <mask>.",
  "C": "W czasie spaceru po kwiecistej łące zrobiłem zdjęcie pięknym <mask>.",
  "B": "W księgarni znalazłam ciekawe <mask>.",
  "N": "Osoby zajmujące się wypiekaniem chleba są z zawodu <mask>.",
  "Msc": "Kibice piłki nożnej rozmawiali o swoich ulubionych <mask>.",
  "W": "<mask>, pomóżcie mi rozwiązać ten problem!"
}

In [71]:
for pl_case, sentence in pl_cases_plural.items():
  print(' >>> ' + pl_case + '. <<<\n')
  test_sequence_with_all_models(sentence)

 >>> M. <<<

 >>> W tym zespole na gitarach grają dwaj przystojni <mask>. <<<

  ROBERTA_words  ROBERTA_scores HERBERT_words  HERBERT_scores POLBERT_words  POLBERT_scores PAPUGAPT_words  PAPUGAPT_scores
0        ludzie        0.660492     mężczyźni        0.313090        muzycy        0.301076        panowie            0.203
1          owie        0.056395       panowie        0.268340       chłopcy        0.109156         muzycy            0.138
2     mężczyźni        0.033644       chłopcy        0.113927     mężczyźni        0.101873      mężczyźni            0.072
3            dj        0.026885    Amerykanie        0.033859        bracia        0.092911    przystojni,            0.067
4            DJ        0.024794        muzycy        0.030317       panowie        0.055067              i            0.054

----------------------------------------------------------------------------------------------------

 >>> D. <<<

 >>> Znana francuska malarka jest autorką tych <mask>. <<<

 

### Liczba mnoga:
- **Mianownik** - forma i sens dla najlepszych dopasowań poprawne
- **Dopełniacz** - forma w większości w porządku, sens też
- **Celownik** - modele mają problem ze znalezieniem właściwej formy - najlepiej radzi sobie Polbert
- **Biernik** - forma i sens dla wszystkich modeli w porządku
- **Narzędnik** - Herbert i Polbert czasami dobrze dopasowują formę wyrazu, gorzej z sensem - nauczyciele i lekarze pieką chleb
- **Miejscownik** - forma i sens właściwie dobrana przez wszystkie modele
- **Wołacz** - dodane wyrazy mają sens i są w poprawnej formie - wołacz jest generalnie dość trudny do przetestowania

Podsumowując - z odmianą przez przypadki najlepiej radzi sobie Polbert, chociaż inne modele nie odstają od niego znacząco, z wyjątkiem PapuGaPT, chociaż w tej sytuacji problemem może być inny rodzaj modelu, który w procedurze testowej nie został idealnie dopasowany do uzupełniania luk.

In [73]:
advanced_grammar = [
    "Ogrodnik rozważał zakup jednej z sadzonek i ostatecznie <mask> krzew z kwiatami w kolorze czerwonym.",
    # sprawdzenie, czy model rozpozna ogrodnika jako podmiot i wstawi czasownik w rodzaju męskim

    "Kapitan przypomina zawodnikom o wyniku meczu, który teraz <mask>.",
    "Kapitan przypomina zawodnikom o meczu, który <mask> w zeszłym roku.",
    "Kapitan przypomina zawodnikom o meczu, który <mask> w przyszłym tygodniu.",
    # sprawdzenie, czy model wykryje konieczność użycia czasu przeszłego i przyszłego

    "Pracownicy zaprotestowali przeciwko szefowi i <mask> wykonania dodatkowej pracy.",
    "Pracownicy pogratulowali szefowi, który <mask> kolejny sukces.",
    # sprawdzenie czy model uzupełni zdanie liczbą mnogą czy pojedynczą

    "Pierwszy algorytm działał wystarczająco szybko, ale drugi okazał się od niego <mask>.",
    "Ze wszystkich sprawdzonych algorytmów ten okazał się <mask>.",
    # sprawdzenie stopniowania przymiotników

    "Rzeźba, która stała na kominku, została <mask> przez złodzieja."
    # sprawdzenie, czy model uzupełni poprawnie stronę bierną
]

for sentence in advanced_grammar:
  test_sequence_with_all_models(sentence)

 >>> Ogrodnik rozważał zakup jednej z sadzonek i ostatecznie <mask> krzew z kwiatami w kolorze czerwonym. <<<

  ROBERTA_words  ROBERTA_scores HERBERT_words  HERBERT_scores POLBERT_words  POLBERT_scores PAPUGAPT_words  PAPUGAPT_scores
0       znalazł        0.209071       powstał        0.209624        wybrał        0.184534                               0
1      stworzył        0.168212      otrzymał        0.164188      otrzymał        0.085839                               0
2         kupił        0.162264        wybrał        0.074842       zakupił        0.074131                               0
3        wybrał        0.156725       uzyskał        0.069272         kupił        0.056911                               0
4      otrzymał        0.104362      stworzył        0.042705    zaoferował        0.029618                               0

----------------------------------------------------------------------------------------------------

 >>> Kapitan przypomina zawodnikom o wynik

## Bardziej skomplikowana gramatyka
- wszystkie modele rozpoznają ogrodnika jako podmiot w dłuższym zdaniu
- modele nie mają problemów z dopasowywaniem formy czasownika do czasu - dość ciekawy jest przypadek zdania:
    `Kapitan przypomina zawodnikom o meczu, który <mask> w zeszłym roku.` - Polbert jako podmiot wstawianego wyrazu wybiera zawodników (wygrali, przegrali itp.), a Herbert kapitana (rozegrał, zagrał) - obydwie formy odpowiedzi mogą być poprawne
- w zdaniach z szefem i pracownikami, które są bardziej precyzyjne wszystkie modele dobrze radzą sobie z liczbą pojedynczą (szef jako podmiot). Z liczbą mnogą nie jest tak dobrze - myli się zwłaszcza Polbert
- testowane modele nie mają też problemów ze stopniowaniem przymiotników
- strona bierna też ok, chociaż tutaj dużo może sugerować słowo "została" znajdujące się bezpośrednio przed luką. Po raz kolejny modele lepiej radzą sobie z gramatyką niż z formułowaniem sensownych odpowiedzi - według Roberty rzeźba została  WYKONANA przez złodzieja – złodziej może w wolnych chwilach zajmować się rzeźbiarstwem, ale nie jest to odpowiedź, której byśmy oczekiwali.


Modele nieźle radzą sobie też z bardziej skomplikowanymi zdaniami, chociaż tym razem Polbert wyróżnia się w sposób negatywny. PapuGaPT nie mógł zostać przetestowany we wszystkich zdaniach, ale tam, gdzie został użyty, wyniki miały sens (chociaż czasami proponowane były słowa, po których ewidentnie miałyby występować jeszcze inne wyrazy).


In [13]:
world_knowledge = [
  "Jedynym metalem występującym w warunkach normalnych w stanie ciekłym jest <mask>.",
  "Największą planetą układu słonecznego jest <mask>.",
  "Narząd, który filtruje krew i usuwa zbędne produkty przemiany materii to <mask>.",
  "Unia Lubelska z 1569 roku miała na celu zintegrowanie administracyjne i polityczne Polski i <mask>.",
  "Głównym bohaterem Lalki Bolesława Prusa jest <mask>.",
  "Twierdzenie Lagrange'a o rozkładach liczb naturalnych mówi, że każda liczba całkowita nieujemna jest sumą kwadratów <mask> liczb całkowitych.",
]

for sentence in world_knowledge:
  test_sequence_with_all_models(sentence)

 >>> Jedynym metalem występującym w warunkach normalnych w stanie ciekłym jest <mask>. <<<

  ROBERTA_words  ROBERTA_scores HERBERT_words  HERBERT_scores POLBERT_words  POLBERT_scores PAPUGAPT_words  PAPUGAPT_scores
0     aluminium        0.125792          woda        0.332970          ołów        0.153272           woda            0.163
1        grafit        0.098311          tlen        0.145733        węgiel        0.145341            wod            0.074
2         metal        0.051446        węgiel        0.112502     aluminium        0.088751         dwutle            0.069
3        magnez        0.031992          ołów        0.063177          cynk        0.059856           tlen            0.064
4        fosfor        0.031531     aluminium        0.026022        żelazo        0.052511            gaz            0.051

----------------------------------------------------------------------------------------------------

 >>> Największą planetą układu słonecznego jest <mask>. <<<



## Wiedza o świecie:
- **Chemia** - żaden model nie  rozpoznaje prawidłowo rtęci
- **Astronomia** - poprawna odpowiedź pojawia się tlko u Rooberty (Jupiter), na piątej pozycji. Herbert wymienia kilka ciał niebieskich, a według Polberta największą planetą jest Meksyk (w pozostałych odpowiedziach również kraje), PapuGaPT wydaje się iść w dobrą stronę (ciała niebieskie), ale odpowiedzi są albo złe, albo niekompletne
- **Anatomia** - jako jedyny nerki  wymienia Herbert (na czwartej pozycji)
- **Historia** - wszystkie modele "wiedzą", że Unia Lubelska wiązała Polskę i Litwę
- **Literatura** - najbardziej szczegółowej odpowiedzi o bohatera "Lalki" udziela Roberta (Stanisław). Taka odpowiedź pojawia się też u PapuGaPT (na piątej pozycji). Inne odpowiedzi nie są jednak zbyt sensowne - główny bohater "Lalki" może i jest człowiekiem i mężczyzną, ale raczej nie ma na imię Piotr, Andrzej czy Mikołaj, nie jest chłopcem, lwem, aktorem ani poetą
- **Matematyka** - Twierdzenie Lagrange'a o rozkładach liczb naturalnych przerasta modele - żaden z nich nie zbliżył się nawet do oczekiwanej odpowiedzi ("sumą kwadratów CZTERECH liczb całkowitych.")

Modele językowe nie posiadają szkolnej wiedzy o świecie - wyjątek stanowi jedynie historyczna informacja o Unii Lubelskiej.

In [9]:
n = 100 # changing to 100 sentences for papugapt to avoid problems with gpu memory

zero_shot_learning = [
    "Książka jest długa i niezbyt ciekawa, bohaterowie schematyczni, a zakończenie rozczarowujące.",
    "Najgorsze firma w jakiej pracowałem. Szef jest złośliwy, a wypłata spóźniona",
    "Hotel nowoczesny i elegancki, pokoje przestronne, ale łóżko bardzo niewygodne. ",
    "Wycieczka, którą mogę śmiało polecić. Zobaczyłem dużo ciekawych miejsc, a jedynym minusem były długie kolejki.",
    "Ta gra planszowa jest świetna. Doskonale bawiłem się przy niej ze znajomymi.",
    "Polecam tą restaurację. Pyszne jedzenie, krótki czas oczekiwania i miła obsługa.",
]

endings = [" - ta wypowiedź ma charakter <mask>.",
           " - ta wypowiedź jest zdecydowanie <mask>.",
           " - sentyment tej odpowiedzi jest <mask>."]

for start in zero_shot_learning:
  for end in endings:
    sentence = start + end
    test_sequence_with_all_models(sentence)

 >>> Książka jest długa i niezbyt ciekawa, bohaterowie schematyczni, a zakończenie rozczarowujące. - ta wypowiedź ma charakter <mask>. <<<

  ROBERTA_words  ROBERTA_scores HERBERT_words  HERBERT_scores POLBERT_words  POLBERT_scores PAPUGAPT_words  PAPUGAPT_scores
0             .        0.110294     krytyczny        0.161112   historyczny        0.111793           humo             0.10
1      powieści        0.100151  informacyjny        0.076748     krytyczny        0.094729         polemi             0.10
2       książki        0.055953      osobisty        0.041460    polityczny        0.077428         czysto             0.08
3             :        0.035860    edukacyjny        0.036232   emocjonalny        0.029308         ogólny             0.06
4             ,        0.027302   symboliczny        0.033757       naukowy        0.025404      wyłącznie             0.05

----------------------------------------------------------------------------------------------------

 >>> Książka 

Chcąc uzyskać jak najlepsze wyniki, każda wypowiedź, dla której miał być wyznaczony sentyment, była łączona z trzema różnymi zakończeniami:
- `- ta wypowiedź ma charakter <mask>.`
- `- ta wypowiedź jest zdecydowanie <mask>.`
- `- sentyment tej odpowiedzi jest <mask>.`


Niestety, niezależnie od sposobu zakończenie zdania, wybrane modele niezbyt dobrze radziły sobie z zero shot learningiem. Najbardziej obiecujące wyniki dawała Roberta - w trzech przykładach była ona w stanie określić sentyment/charakter wypowiedzi jako "pozytywny", ale w większości zwracane przez nią słowa nie były tym, czego oczekiwaliśmy. W odpowiedziach innych modeli sporadycznie pojawiały się słowa, które można by wykorzystać do analizy sentymentu ("pozytywna", "negatywny", "krytyczny"), ale nie były one na pierwszych miejscach i żaden model nie wyróżniał się pozytywnie na tle pozostałych. Prawdopodobnie wybrane zdania okazały się zbyt długie, aby móc wykorzystać w ten sposób możliwości badanych modeli.


In [ ]:
! zip -r /content/results.zip /content/results/

## Wnioski:
- w przypadku badanych modeli można bardziej liczyć na udzielenie poprawnej gramatycznie odpowiedzi, niż na wybranie słowa, które będzie miało sens w kontekście pozostałych wyrazów w zdaniu.

- w zadaniach dotyczących wiedzy o świecie oraz zero shot learningu testowane modele radzą sobie słabo 

- modele językowe dobrze jest stosować do takich celów, do jakich zostały przygotowane – model PapuGaPT2 stworzony do generowanie tekstu, a nie uzupełniania luk, nie mógł być łatwo porównany z innymi modelami. Nie mógł on zostać użyty w zdaniach, w których uzupełniane słowo nie znajdowało się na końcu. Również ograniczenie na generowanie jednego dodatkowego tokenu sprawiło, że w niektórych zadaniach model zwracał jedynie początki wyrazów. 

- żaden z pozostałych modeli (Roberta, Herbert i Polbert) nie okazał się w przeprowadzonych testach wyraźnie lepszy od swoich konkurentów, chociaż wyniki w zadaniach były mocno zróżnicowane. W odmianie przez przypadki najlepiej wypadł Polbert, który jednak całkowicie mylił się w jednym z bardziej skomplikowanych zdań. Polbert bardzo źle wypadł również w teście wiedzy o świecie, częściej niż inne modele dopasowując do zdań słowa, które w ogóle nie pasowały do kategorii pytania. Roberta, która z odmianą przez przypadki radziła sobie najgorzej, nieco lepiej od konkurentów wypadła w zadaniu dotyczącym wiedzy o świecie i zero shot learningu. 
